# Task 2: Text Chunking, Embedding, and Vector Store Indexing

In [3]:
import sys
import os
import pandas as pd
from glob import glob

In [4]:
from pathlib import Path
from importlib import reload
# add the project root to the path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

### Load the dataset

In [5]:
# import the module to load the data into dataframes
from src.data_loader import DataLoader
import src.data_loader
reload(src.data_loader)

<module 'src.data_loader' from '/home/chalasimon/Documents/10academy/week 6/challenge/CrediTrust-RAG-Chatbot/src/data_loader.py'>

In [6]:
# Load the dataset
data_loader = DataLoader('../data/processed/filtered_complaints.csv')
df = data_loader.load_data()
print(f"Loaded {len(df)} cleaned complaints")

/home/chalasimon/Documents/10academy/week 6/challenge/CrediTrust-RAG-Chatbot/src/data_loader.py:20: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(self.file_path)


Loaded 9609797 cleaned complaints


In [7]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,narrative_length
0,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,Experian Information Solutions Inc.,FL,32092,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195687,0
1,2025-06-20,Debt collection,Telecommunications debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company can't verify or dispute the facts in t...,"Eastern Account Systems of Connecticut, Inc.",FL,342XX,NaN,NaN,Web,2025-06-20,Closed with explanation,Yes,NaN,14195688,0
2,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AZ,85225,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195689,0
3,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,Experian Information Solutions Inc.,AZ,85225,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195690,0
4,2025-06-20,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,Experian Information Solutions Inc.,IL,60628,NaN,NaN,Web,2025-06-20,In progress,Yes,NaN,14195692,0


### Text Chuncking

In [8]:
# Import the text splitter and initialize it
from src.embedding.chunking import get_text_splitter
text_splitter = get_text_splitter()

In [ ]:
from tqdm import tqdm
# chunking process
docs = []
metadata = []

for _, row in df.iterrows():
    chunks = text_splitter.split_text(row['clean_text'])
    for chunk in chunks:
        docs.append(chunk)
        metadata.append({
            "complaint_id": row['Complaint ID'],
            "product": row['MappedProduct']
        })

print(f"Total text chunks created: {len(docs)}")